In [1]:
%reload_ext autoreload
%autoreload 2
import numpy as np, sys, os, pandas as pd,json, os
sys.path.insert(1, '../../')
from getting_data import load_sample, feature_key_list, categorical_feature_key_list
from s2search_score_pipelining import get_scores
from s2search_score_pdp import save_pdp_to_npz

setting = 0
exp_name = 'exp5'
sample_name = 'cslg'

query = 'Machine Learning'

categorical_name = {}

def remove_duplicate(seq):
    seen = set()
    seen_add = seen.add
    return [x for x in seq if not (x in seen or seen_add(x))]

categorical_name_file = os.path.join('.', 'scores', f'{sample_name}_categorical_name.npz')

for i in range(len(feature_key_list)):
    feature_name = feature_key_list[i]
    if feature_name in categorical_feature_key_list:
        df = load_sample(exp_name, sample_name, query=query, sort=feature_name, rank_f=get_scores)
        if feature_name == 'authors':
            l = [json.dumps(x) for x in df[feature_name]]
        else:
            l = list(df[feature_name])
        categorical_name[i] = remove_duplicate(l)

fail to not force global because 1 worker available
get ranker in 7920 with global setting: True and gb_ranker len 1
[Main taks:-1] compute 92938 scores with worker 7920
[Main taks][03/24/2022, 00:09:31] 92938 scores within 35.520907 sec 
fail to not force global because 1 worker available
get ranker in 7920 with global setting: True and gb_ranker len 1
[Main taks:-1] compute 92938 scores with worker 7920
[Main taks][03/24/2022, 00:10:23] 92938 scores within 46.723759 sec 
fail to not force global because 1 worker available
get ranker in 7920 with global setting: True and gb_ranker len 1
[Main taks:-1] compute 92938 scores with worker 7920
[Main taks][03/24/2022, 00:10:43] 92938 scores within 15.141943 sec 
fail to not force global because 1 worker available
get ranker in 7920 with global setting: True and gb_ranker len 1
[Main taks:-1] compute 92938 scores with worker 7920
[Main taks][03/24/2022, 00:11:13] 92938 scores within 25.299571 sec 


In [2]:
df = load_sample(exp_name, sample_name)
paper_data = json.loads(df.to_json(orient='records'))

y_pred_file = os.path.join('.', 'scores', f'{sample_name}_y_pred.npz')
if not os.path.exists(y_pred_file):
    y_pred = get_scores(query, paper_data)
    save_pdp_to_npz('.', y_pred_file, y_pred)
else:
    y_pred = np.load(y_pred_file)['arr_0']

In [3]:
def get_class(score):
    if score <= -17:
        return '(,-17]'
    elif score <= -10:
        return '(-17, -10]'
    elif score <= -5:
        return '(-10, -5]'    
    elif score <= 0:
        return '(-5, <0]'  
    elif score <= 3:
        return '(0, 3]'
    elif score <= 5:
        return '(3, 5]'
    elif score <= 6:
        return '(5, 6]'
    elif score <= 7:
        return '(6, 7]'
    elif score <= 8:
        return '(7, 8]'
    elif score <= 9:
        return '(8, 9]'
    else:
        return '(9,)'

# make class_name
class_name = ['(,-17]','(-17, -10]','(-10, -5]','(-5, <0]','(0, 3]','(3, 5]','(5, 6]','(6, 7]','(7, 8]','(8, 9]','(9,)']

categorical_name_map = {}

for i in range(len(feature_key_list)):
    feature_name = feature_key_list[i]
    if feature_name in categorical_feature_key_list:
        categorical_name_map[i] = {}
        values = categorical_name[i]
        for j in range(len(values)):
            value = values[j]
            categorical_name_map[i][value] = j

# encoding data
for i in range(len(paper_data)):
    paper_data[i] = [
        categorical_name_map[0][paper_data[i]['title']], categorical_name_map[1][paper_data[i]['abstract']],
        categorical_name_map[2][paper_data[i]['venue']], categorical_name_map[3][json.dumps(paper_data[i]['authors'])],
        paper_data[i]['year'],
        paper_data[i]['n_citations']
    ]

In [4]:
def decode_paper(encoded_p):
    return dict(
        title=categorical_name[0][encoded_p[0]],
        abstract=categorical_name[1][encoded_p[1]],
        venue=categorical_name[2][encoded_p[2]],
        authors=json.loads(categorical_name[3][encoded_p[3]]),
        year=encoded_p[4],
        n_citations=encoded_p[5],
    )

decoded_i = 92937

print(paper_data[decoded_i])
print(df.iloc[decoded_i])
print(decode_paper(paper_data[decoded_i]))

[55397, 28139, 2995, 84492, 2019, 61]
title          Pricing options and computing implied volatili...
abstract       This paper proposes a data-driven approach, by...
venue                                                      Risks
authors        [Shuaiqiang  Liu, Cornelis W. Oosterlee, Sande...
year                                                        2019
n_citations                                                   61
Name: 92937, dtype: object
{'title': 'Pricing options and computing implied volatilities using neural networks', 'abstract': 'This paper proposes a data-driven approach, by means of an Artificial Neural Network (ANN), to value financial options and to calculate implied volatilities with the aim of accelerating the corresponding numerical methods. With ANNs being universal function approximators, this method trains an optimized ANN on a data set generated by a sophisticated financial model, and runs the trained ANN as an agent of the original solver in a fast and eff

In [5]:
from anchor import anchor_tabular

paper_data = np.array(paper_data)

explainer = anchor_tabular.AnchorTabularExplainer(
    class_name,
    feature_key_list,
    paper_data,
    categorical_name)

In [37]:
def pred_fn(x):
    predictions = get_scores(query, [decode_paper(p) for p in x], ptf = False)
    encoded_pred = [class_name.index(get_class(pp)) for pp in predictions]
    return np.array(encoded_pred)

idx = 92936
exp = explainer.explain_instance(paper_data[idx], pred_fn, threshold=0.9999)

In [38]:
print('Prediction: ', explainer.class_names[pred_fn([paper_data[idx]])[0]])
print('-------------------')
print('Anchor: \n%s' % ('\nAND\n'.join(exp.names())))
print('\nPrecision: %.2f' % exp.precision())
print('Coverage: %.2f' % exp.coverage())

Prediction:  (6, 7]
-------------------
Anchor: 
abstract = With the growing interest in machine learning community to solve real world problems, it has become crucial to uncover the hidden reasoning behind their decisions by focusing on the fairness and auditing the predictions made by these black-box models. In this paper, we propose a novel method to address two key issues: (a) Can we simultaneously learn fair disentangled representations while ensuring the utility of the learned representation for downstream tasks, and (b) Can we provide theoretical insights into when the proposed approach will be both fair and accurate. To address the former, we propose the method FRIED, Fair Representation learning using Interpolation Enabled Disentanglement. In our architecture, by imposing a critic-based adversarial framework, we enforce the interpolated points in the latent space to be more realistic. This helps in capturing the data manifold effectively and enhances utility of the learned rep

In [40]:
metrics = dict(
    title=[],
    abstract=[],
    venue=[],
    authors=[],
    year=[],
    n_citations=[],
)
previous_single_precision = 0

for j in range(len(exp.names())):
    name = exp.names()[j]
    print(name)
    current_single_precision = exp.precision(j) - previous_single_precision
    previous_single_precision = exp.precision(j)
    for feature_name in metrics.keys():
        if name.startswith(f'{feature_name} = '):
            metrics[feature_name].append(current_single_precision)

metrics

abstract = With the growing interest in machine learning community to solve real world problems, it has become crucial to uncover the hidden reasoning behind their decisions by focusing on the fairness and auditing the predictions made by these black-box models. In this paper, we propose a novel method to address two key issues: (a) Can we simultaneously learn fair disentangled representations while ensuring the utility of the learned representation for downstream tasks, and (b) Can we provide theoretical insights into when the proposed approach will be both fair and accurate. To address the former, we propose the method FRIED, Fair Representation learning using Interpolation Enabled Disentanglement. In our architecture, by imposing a critic-based adversarial framework, we enforce the interpolated points in the latent space to be more realistic. This helps in capturing the data manifold effectively and enhances utility of the learned representation for downstream prediction tasks. We a

{'title': [0.6061285562361398],
 'abstract': [0.21565362198168192],
 'venue': [],
 'authors': [],
 'year': [],
 'n_citations': []}

In [8]:
import datetime
import pytz
utc_tz = pytz.timezone('America/Montreal')

metrics = dict(
    title=0,
    abstract=0,
    venue=0,
    authors=0,
    year=0,
    n_citations=0,
)

for i in range(len(paper_data)):
    print(i, datetime.datetime.now(tz=utc_tz).strftime("%m/%d/%Y, %H:%M:%S"))
    exp = explainer.explain_instance(paper_data[i], pred_fn, threshold=0.99, tau=0.5)

    previous_single_precision = 0
    for j in range(len(exp.names())):
        name = exp.names()[j]
        current_single_precision = exp.precision(j) - previous_single_precision
        previous_single_precision = exp.precision(j)
        for feature_name in metrics.keys():
            if name.startswith(f'{feature_name} = '):
                metrics[feature_name] += (current_single_precision)

    break
metrics

0 03/24/2022, 00:11:22


{'title': 0.27368109945322894,
 'abstract': 0.7164179104477612,
 'venue': 0,
 'authors': 0,
 'year': 0,
 'n_citations': 0}